In [2]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(palette='Set2')
import plotly.graph_objects as go

import scipy.stats as ss

import warnings
warnings.filterwarnings('ignore')

import nbimporter

## Auxilary Functions

In [3]:
def estimate_SR(prob: float, sl: float, pt: float, freq: float, num_trials: int = 1000000) -> float:
    '''
    Estimates strategy's Sharpe ratio under given parameters.
    
        Parameters:
            prob (float): precision of the strategy
            sl (float): stop loss threshold
            pt (float): profit taking threshold
            freq (float): annual number of bets (to obtain annualized SR)
            num_trial (int): number of trials used for estimation
            
        Returns:
            sr (float): Sharpe ratio
    '''
    out = []
    for i in range(num_trials):
        rnd = np.random.binomial(n=1, p=prob)
        if rnd == 1:
            x = pt
        else:
            x = sl
        out.append(x)
    sr = np.mean(out) / np.std(out) * np.sqrt(freq)
    return sr

In [4]:
def bin_HR(sl: float, pt: float, freq: float, tSR: float) -> float:
    '''
    Returns minimum precision p needed to achieve target Sharpe ration under given parameters.
    
        Parameters:
            sl (float): stop loss threshold
            pt (float): profit taking threshold
            freq (float): annual number of bets
            tSR (float): target annual Sharpe ratio
            
        Returns:
            p (float): precision
    '''
    a = (freq + tSR ** 2) * (pt - sl) ** 2
    b = (2 * freq * sl - tSR ** 2 * (pt - sl)) * (pt - sl)
    c = freq * sl ** 2
    p = (-b + (b ** 2 - 4 * a * c) ** 0.5) / (2.0 * a)
    return p

In [5]:
def bin_freq(sl: float, pt: float, p: float, tSR: float) -> float:
    '''
    Returns minimum number of bets per year needed to achieve target Sharpe ration under given parameters.
    
        Parameters:
            sl (float): stop loss threshold
            pt (float): profit taking threshold
            p (float): precision
            tSR (float): target annual Sharpe ratio
            
        Returns:
            freq (float): annual number of bets
    '''
    freq = (tSR * (pt - sl)) ** 2 * p * (1 - p) / ((pt - sl) * p + sl) ** 2
    return freq

In [6]:
def mix_gaussians(
    mu1: float, mu2: float, sigma1: float, sigma2: float, prob1: float, nObs: int
) -> np.ndarray:
    '''
    Generates random draws form a mixture of two Gaussians.
    
        Parameters:
            mu1 (float): expectation of 1st Gaussian
            mu2 (float): expectation of 2nd Gaussian
            sigma1 (float): std of 1st Gaussian
            sigma2 (float): std of 2nd Gaussian
            prob1 (float): probability of generating from 1st Gaussian (i.e. weight of 1st Gaussian)
            nObs (int): total number of draws
            
        Returns:
            ret (np.ndarray): array with observations
    '''
    ret1 = np.random.normal(mu1, sigma1, size=int(nObs * prob1))
    ret2 = np.random.normal(mu2, sigma2, size=nObs - ret1.shape[0])
    ret = np.append(ret1, ret2, axis=0)
    np.random.shuffle(ret)
    return ret

In [7]:
def prob_failure(ret: np.ndarray, freq: float, tSR: float):
    '''
    Derives probability that strategy has lower precision than needed.
    
        Parameters:
            ret (np.ndarray): array with observations
            freq (float): annual number of bets
            tSR (float): target Sharpe ratio
            
        Returns:
            risk (float): probability of failure
    '''
    rPos, rNeg = ret[ret > 0].mean(), ret[ret <= 0].mean()
    p = ret[ret > 0].shape[0] / float(ret.shape[0])
    thresP = bin_HR(rNeg, rPos, freq, tSR)
    risk = ss.norm.cdf(thresP, p, p * (1 - p))    # approximation to bootstrap
    return risk

## Exercises

### 1. Evaluating strategy.

**Q:** A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit-taking, and –2% for stop-loss.

1. Is this strategy viable?

In [8]:
print(f'Estimated SR of the strategy: {estimate_SR(prob=0.6, pt=0.02, sl=-0.02, freq=52)}')

Estimated SR of the strategy: 1.4781562099865402


We got a much lower SR that targeted.

2. What is the required precision rate that would make the strategy profitable?

In [9]:
print(f'Required precision: {"{:.3f}".format(bin_HR(sl=-0.02, pt=0.02, freq=52, tSR=2.0))}')

Required precision: 0.634


3. For what betting frequency is the target achievable?

In [10]:
print(f'Required frequency: {bin_freq(sl=-0.02, pt=0.02, p=0.6, tSR=2.0)}')

Required frequency: 96.0


4. For what profit-taking threshold is the target achievable?

In [11]:
for pt in np.linspace(0.02, 0.03, 100):
    if estimate_SR(prob=0.6, pt=pt, sl=-0.02, freq=52, num_trials=100000) >= 2.0:
        break
print(f'Required profit taking threshold: {"{:.3f}".format(pt)}')

Required profit taking threshold: 0.023


5. What would be an alternative stop-loss?

In [12]:
for sl in np.linspace(0.02, 0.01, 100):
    if estimate_SR(prob=0.6, pt=0.02, sl=-sl, freq=52, num_trials=100000) >= 2.0:
        break
print(f'Required stop loss threshold: {"{:.3f}".format(-sl)}')

Required stop loss threshold: -0.017


### 2. More Evaluation.

Here I assume that precision, frequency, and profit taking threshold are increased by 1%, and stop loss threshold is decreased by 1% in absolute value.

In [13]:
old_sr = estimate_SR(prob=0.6, pt=0.02, sl=-0.02, freq=52)
new_sr_prob = estimate_SR(prob=0.6 * 1.01, pt=0.02, sl=-0.02, freq=52)
new_sr_freq = estimate_SR(prob=0.6, pt=0.02, sl=-0.02, freq=52 * 1.01)
new_sr_pt = estimate_SR(prob=0.6, pt=0.02 * 1.01, sl=-0.02, freq=52)
new_sr_sl = estimate_SR(prob=0.6, pt=0.02, sl=-0.02 * 0.99, freq=52)

for parameter, new_sr in zip(['precision', 'frequency', 'profit taking', 'stop loss'],
                             [new_sr_prob, new_sr_freq, new_sr_pt, new_sr_sl]):
    print(f'SR sensitivity to 1% change in {parameter}: {"{:.2f}".format(100 * (new_sr / old_sr - 1))}%')

SR sensitivity to 1% change in precision: 6.76%
SR sensitivity to 1% change in frequency: 0.60%
SR sensitivity to 1% change in profit taking: 3.19%
SR sensitivity to 1% change in stop loss: 2.59%


It appears that SR is most sensitive to changes in precision.

### 3. Probability of Failure.

Here we compute the probability that the strategy has precision lower than necessary for achieving Sharpe ratio of 1.

In [14]:
ret = []
for i in range(1000000):
    rnd = np.random.binomial(n=1, p=0.6)
    if rnd == 1:
        x = 0.02
    else:
        x = -0.02
    ret.append(x)

print(f'Probability of failure: {"{:.3f}".format(prob_failure(ret=np.array(ret), freq=52, tSR=1.0))}')

Probability of failure: 0.447


We can also compute PSR introduced in Chapter 14:

In [15]:
skewness = ss.skew(ret)
kurt = ss.kurtosis(ret)
sr = np.mean(ret) / np.std(ret)
val = (sr - 1.0) * np.sqrt(len(ret) - 1) / np.sqrt(1 - skewness * sr + (kurt - 1) / 4 * sr ** 2)
psr = ss.norm.cdf(val)
print(f'Probability of achieving targeted SR: {psr}')

Probability of achieving targeted SR: 0.0
